# SAM 3D Mesh 추출 튜토리얼

이미지에서 3D Mesh를 추출하는 최소 코드

## 1. Setup

In [ ]:
import sys
import numpy as np
from PIL import Image

# SAM 3D 모듈 경로 추가
sys.path.append("/home/joon/dev/sam3d_gui/external/sam-3d-objects/notebook")
from inference import Inference

## 2. 모델 로드

In [ ]:
# Checkpoint 경로
CHECKPOINT_PATH = "/home/joon/dev/sam3d_gui/checkpoints/sam3d/hf"
config_path = f"{CHECKPOINT_PATH}/pipeline.yaml"

# 모델 로드 (최초 1회만 실행)
model = Inference(config_path, compile=False)
print("✓ 모델 로드 완료")

## 3. 이미지 & 마스크 로드

In [ ]:
# 이미지 경로 (변경 필요)
IMAGE_PATH = "/path/to/your/image.png"
MASK_PATH = "/path/to/your/mask.png"  # Optional: 없으면 전체 이미지 사용

# 이미지 로드
image = np.array(Image.open(IMAGE_PATH).convert("RGB"))

# 마스크 로드 (흰색=객체, 검정=배경)
mask = np.array(Image.open(MASK_PATH).convert("L"))

# RGBA 이미지 생성 (SAM3D 입력 형식)
rgba = np.concatenate([image, mask[..., None]], axis=-1)
print(f"RGBA shape: {rgba.shape}")

## 4. 3D 추론 실행

In [ ]:
# 추론 실행
output = model._pipeline.run(
    rgba,
    None,  # mask (이미 RGBA에 포함)
    seed=42,
    with_mesh_postprocess=True,   # Mesh 후처리 (FlexiCubes)
    with_texture_baking=False,    # 텍스처 베이킹 (느림, 선택적)
    use_vertex_color=True,
)

print(f"✓ 추론 완료")
print(f"Output keys: {output.keys()}")

## 5. Mesh 추출 & 저장

In [ ]:
import trimesh

OUTPUT_PATH = "/path/to/output/mesh.ply"

# 방법 1: GLB (후처리된 mesh, 권장)
if 'glb' in output and output['glb'] is not None:
    mesh = output['glb']  # trimesh 객체
    print(f"GLB mesh: {len(mesh.vertices)} vertices, {len(mesh.faces)} faces")
    mesh.export(OUTPUT_PATH)
    print(f"✓ 저장 완료: {OUTPUT_PATH}")

# 방법 2: Raw mesh (후처리 없이)
elif 'mesh' in output and output['mesh'] is not None:
    mesh_result = output['mesh'][0]  # MeshExtractResult
    vertices = mesh_result.vertices.detach().cpu().numpy()
    faces = mesh_result.faces.detach().cpu().numpy()
    print(f"Raw mesh: {len(vertices)} vertices, {len(faces)} faces")
    
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
    mesh.export(OUTPUT_PATH)
    print(f"✓ 저장 완료: {OUTPUT_PATH}")

# 방법 3: Gaussian Splatting (point cloud)
elif 'gs' in output and output['gs'] is not None:
    gs_path = OUTPUT_PATH.replace('.ply', '_gaussian.ply')
    output['gs'].save_ply(gs_path)
    print(f"⚠ Mesh 없음, Gaussian 저장: {gs_path}")

## 6. 전체 파이프라인 (함수화)

In [ ]:
def extract_3d_mesh(
    image_path: str,
    mask_path: str,
    output_path: str,
    model: Inference,
    with_texture: bool = False,
    seed: int = 42
):
    """
    이미지에서 3D mesh 추출
    
    Args:
        image_path: RGB 이미지 경로
        mask_path: 마스크 이미지 경로 (흰색=객체)
        output_path: 출력 mesh 경로 (.ply, .glb, .obj)
        model: SAM3D Inference 모델
        with_texture: 텍스처 베이킹 활성화 (느림)
        seed: 랜덤 시드
    """
    import trimesh
    
    # 1. 이미지 로드
    image = np.array(Image.open(image_path).convert("RGB"))
    mask = np.array(Image.open(mask_path).convert("L"))
    rgba = np.concatenate([image, mask[..., None]], axis=-1)
    
    # 2. 추론
    output = model._pipeline.run(
        rgba, None, seed=seed,
        with_mesh_postprocess=True,
        with_texture_baking=with_texture,
        use_vertex_color=True,
    )
    
    # 3. Mesh 추출
    if 'glb' in output and output['glb'] is not None:
        mesh = output['glb']
    elif 'mesh' in output and output['mesh'] is not None:
        m = output['mesh'][0]
        mesh = trimesh.Trimesh(
            vertices=m.vertices.detach().cpu().numpy(),
            faces=m.faces.detach().cpu().numpy()
        )
    else:
        raise ValueError("No mesh in output")
    
    # 4. 저장
    mesh.export(output_path)
    print(f"✓ {len(mesh.vertices)} vertices, {len(mesh.faces)} faces -> {output_path}")
    return mesh

In [ ]:
# 사용 예시
mesh = extract_3d_mesh(
    image_path="/path/to/image.png",
    mask_path="/path/to/mask.png",
    output_path="/path/to/output.ply",
    model=model,
    with_texture=False
)